In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import re
import emoji
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    Adafactor, get_linear_schedule_with_warmup
)
from sentence_transformers import SentenceTransformer,  InputExample, LoggingHandler
from xgboost import XGBClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
import pickle

In [ ]:
data_train = pd.read_csv('/content/drive/MyDrive/Znatno_personal/Отчетность ВКР/Артефакты/Parsing_avito/Model for benchmark/data/train - train (3).csv')

In [ ]:
data_test = pd.read_csv('/content/drive/MyDrive/Znatno_personal/Отчетность ВКР/Артефакты/Parsing_avito/Model for benchmark/data/test_data.csv')

In [ ]:
X = data_train['message.text']
y = data_train['Оценка']
X_test = data_test['message.text']
y_test = data_test['Оценка']

#Предобработка

In [ ]:
def convert_emojis_to_words(text):

    # Convert emojis to words
    text = emoji.replace_emoji(text, replace="")
    # Remove the : from the words and replace _ with space
    text = text.replace("_", " ")

    return text

In [ ]:
symbols_pattern = re.compile(pattern = "["
    "@_!#$%^&*()<>?/\|}{~√•—"
                       "]+", flags = re.UNICODE) #спецсимволы
# двойные пробелы
space_pattern = re.compile('\s+')

In [ ]:
def clear_text(text):
    """ Функция удаления спецсимволов"""
    # удаление спецсимволов и emoji
    pre = symbols_pattern.sub(r'',text)
    pre = re.sub(r'http\S+', '', pre)
    pre = convert_emojis_to_words(pre)

    return space_pattern.sub(' ', pre)

In [ ]:
def preprocess_text(text):
    """ Финальная функция для обработки """
    # srip + lower + punctuation
    sentence = (
        ''.join([x for x in str(text).strip().lower()])
    )

    return clear_text(sentence)

In [ ]:
data_train['message.text'] = data_train['message.text'].apply(preprocess_text)

In [ ]:
X = X.apply(preprocess_text)

In [ ]:
X_test = X_test.apply(preprocess_text)

In [ ]:
df_train, df_val = train_test_split(data_train, test_size=0.1, random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Fine tuning Sentence Transformer

In [ ]:
model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

In [ ]:
class ClassificationHead(nn.Module):
  def __init__(self, embedding_dim, num_classes):
    super(ClassificationHead, self).__init__()
    self.linear = nn.Linear(embedding_dim, num_classes)
  def forward(self, data):
    x = data['sentence_embedding']
    x = self.linear(x)
    return x

num_labels = 9
classification_head = ClassificationHead(model.get_sentence_embedding_dimension(), num_labels)

In [ ]:
class SentenceTransformerplusHead(nn.Module):
  def __init__(self, transformer, classification_head):
    super(SentenceTransformerplusHead, self).__init__()
    self.transformer = transformer
    self.classification_head = classification_head
  def forward(self, input):
    data = self.transformer(input)
    logits = self.classification_head(data)
    return logits

In [ ]:
sentence_transformer_plus_head = SentenceTransformerplusHead(model, classification_head)

In [ ]:
# training parameters
num_epochs = 100
batch_size = 32
learning_rate = 1e-3


In [ ]:
# Convert the dataset to PyTorch tensors.
train_examples = [InputExample(texts=[s], label=l) for s, l in zip(X, y)]

# Customize collate_fn to convert InputExample objects into tensors.
def collate_fn(batch):
    texts = [example.texts[0] for example in batch]
    labels = torch.tensor([example.label for example in batch])
    return texts, labels

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size, collate_fn=collate_fn)

# Define the loss function, optimizer, and learning rate scheduler.
criterion = nn.CrossEntropyLoss()
optimizer = Adafactor(sentence_transformer_plus_head.parameters(), relative_step=False, lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
loss_list=[]
for epoch in range(num_epochs):
    sentence_transformer_plus_head.train()
    for step, (texts, labels) in enumerate(train_dataloader):
        labels = labels.to(model.device)
        optimizer.zero_grad()

        # Encode text and pass through classification head.
        inputs = model.tokenize(texts)
        input_ids = inputs['input_ids'].to(model.device)
        input_attention_mask = inputs['attention_mask'].to(model.device)
        inputs_final = {'input_ids': input_ids, 'attention_mask': input_attention_mask}

        # move model_with_head to the same device
        sentence_transformer_plus_head = sentence_transformer_plus_head.to(model.device)
        logits = sentence_transformer_plus_head(inputs_final)

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        if step % 500 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item()}")

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
    model_save_path = '//content/drive/MyDrive/finetuning_sentance_transformer_plus_head11/'
    model.save(model_save_path)
    loss_list.append(loss.item())

In [ ]:
# Save the final model
model_final_save_path='//content/drive/MyDrive/finetuning_sentance_transformer_plus_head/st_ft1_epoch100'
model.save(model_final_save_path)

In [ ]:
y_train = np.array(y)
y_test = np.array(y_test)

In [ ]:
X_train

# Обучение и тестирование XGBoost

In [ ]:
model = SentenceTransformer('//content/drive/MyDrive/finetuning_sentance_transformer_plus_head/st_ft_epoch100')

In [ ]:
train_embeddings  = X_train.apply(lambda x: model.encode(str(x))).to_list()
val_embeddings  = X_val.apply(lambda x: model.encode(str(x))).to_list()
test_embeddings  = X_test.apply(lambda x: model.encode(str(x))).to_list()

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [ ]:
train_embeddings = np.array(train_embeddings)
val_embeddings = np.array(val_embeddings)
test_embeddings = np.array(test_embeddings)

In [ ]:
xgb1 = XGBClassifier(objective="multi:softmax", num_class=9, eval_metric="mlogloss", learning_rate=0.07078025811321921, max_depth=45, n_estimators = 700, subsample=0.5458605221291573, colsample_bytree= 0.15268221810604315)
# Предсказания и оценка
xgb1.fit(train_embeddings, y_train,eval_set= [(train_embeddings, y_train), (val_embeddings, y_val)])

y_pred = xgb1.predict(test_embeddings)

In [ ]:
evals_result = xgb1.evals_result()

In [ ]:
print("Accuracy :", accuracy_score(y_test, y_pred))
print("F1 score:",f1_score(y_test, y_pred, average='weighted'))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))

fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(cm, interpolation='nearest', aspect='auto')
unique_labels = np.unique(y_test).astype(int)
ax.set_xticks(range(9))
ax.set_yticks(range(9))
ax.set_xticklabels(unique_labels, rotation=45)
ax.set_yticklabels(unique_labels)
ax.set_ylabel('Истинный класс')
ax.set_xlabel('Предсказанный класс')
fig.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()
plt.savefig('xbg_matrix.png')


In [ ]:
# Сначала получим список всех классов
classes = np.unique(y_test)

# 1) F1-score по классам
f1_per_class = f1_score(y_test, y_pred, labels=classes, average=None, zero_division=0)

# 2) «Точность по классу» как доля правильно предсказанных среди всех примеров этого класса
#    (этот показатель в мультиклассе эквивалентен recall для данного класса)
accuracy_per_class = np.array([
    np.mean(y_pred[y_test == cls] == cls)
    for cls in classes
])

# 3) Собираем всё в DataFrame
df = pd.DataFrame({
    'class': classes.astype(str),
    'accuracy': accuracy_per_class,
    'f1_score': f1_per_class
}).set_index('class')

print(df)



# Оптимизация гиперпараметров XGBoost и повторное обучение

In [ ]:
def objective(trial):
    params = {
        "objective": "multi:softmax",
        "num_class": 9,
        "eval_metric": "mlogloss",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.0001, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 100),
        "n_estimators": trial.suggest_int("n_estimators", 50, 2000),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0)
    }

    model = XGBClassifier(**params)
    model.fit(train_embeddings, y_train)

    y_pred = model.predict(val_embeddings)
    return accuracy_score(y_val, y_pred)

# Запуск оптимизации
study = optuna.create_study(direction="maximize")  # Ищем max accuracy
study.optimize(objective, n_trials=50)

# Лучшие параметры
print("Лучшие параметры:", study.best_params)

In [ ]:
xgb2 =XGBClassifier(objective="multi:softmax", num_class=9, eval_metric="mlogloss", learning_rate=0.005571440502204253, max_depth=11, n_estimators = 140, subsample=0.6473257042288756, colsample_bytree = 0.14967908315900308)

In [ ]:
xgb2.fit(train_embeddings, y_train,eval_set= [(train_embeddings, y_train), (val_embeddings, y_val)])

In [ ]:
y_pred = xgb2.predict(test_embeddings)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.5f}')

In [ ]:
print("Accuracy :", accuracy_score(y_test, y_pred))
print("F1 score:",f1_score(y_test, y_pred, average='weighted'))

In [ ]:
classes = np.unique(y_test)

# 1) F1-score по классам
f1_per_class = f1_score(y_test, y_pred, labels=classes, average=None, zero_division=0)

# 2) «Точность по классу» как доля правильно предсказанных среди всех примеров этого класса
#    (этот показатель в мультиклассе эквивалентен recall для данного класса)
accuracy_per_class = np.array([
    np.mean(y_pred[y_test == cls] == cls)
    for cls in classes
])

# 3) Собираем всё в DataFrame
df = pd.DataFrame({
    'class': classes.astype(str),
    'accuracy': accuracy_per_class,
    'f1_score': f1_per_class
}).set_index('class')

print(df)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))

fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(cm, interpolation='nearest', aspect='auto')
unique_labels = np.unique(y_test).astype(int)
ax.set_xticks(range(9))
ax.set_yticks(range(9))
ax.set_xticklabels(unique_labels, rotation=45)
ax.set_yticklabels(unique_labels)
ax.set_ylabel('Истинный класс')
ax.set_xlabel('Предсказанный класс')
fig.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()
plt.savefig('xbg_matrix.png')

# Формирование файла Pickle

In [ ]:
class Preprocessing:
    def __init__(self):
        """
        :param text: исходная строка для предварительной обработки
        """
        pass
    def fit(self, text,y=None):
      return self

    def make_string(self,text):
        """
        Приводит текст к строке, удаляет лишние пробелы и приводит к нижнему регистру
        """
        return str(text).strip().lower()

    def convert_emojis_to_words(self, text=None):
        """
        Преобразует emojis в слова
        :param text: текст для обработки; если None, используется self.text
        """
        t = text if text is not None else text
        # Переводим emoji в текстовый формат :emoji_name:
        text_1 = emoji.demojize(str(t), delimiters=(" ", " "))
        # Удаляем двоеточия и заменяем подчёркивания на пробелы
        text_1 = text_1.replace(':', '').replace('_', ' ')
        return text_1

    def clear_text(self, text):
        """
        Удаляет спецсимволы и нормализует пробелы
        :param text: текст для очистки
        """
        symbols_pattern = re.compile(r"[@_!#$%^&*()<>?/\\|}{~√•—]+", flags=re.UNICODE)
        space_pattern = re.compile(r"\s+")
        # Удаляем спецсимволы
        pre = symbols_pattern.sub('', text)
        # Нормализуем пробелы
        return space_pattern.sub(' ', pre).strip()

    def transform(self,text):
        """
        Полная предварительная обработка текста:
        - stringify + lower
        - convert emojis
        - clear text
        :returns: обработанная строка
        """
        s = self.make_string(text)
        s = self.convert_emojis_to_words(s)
        s = self.clear_text(s)
        return s



In [ ]:
class Get_embeddings:
  def __init__(self):
    self.model = SentenceTransformer('//content/drive/MyDrive/finetuning_sentance_transformer_plus_head/st_ft_epoch100')
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    train_embeddings  = list(model.encode(X))
    return train_embeddings

In [ ]:
pipe = Pipeline([('get_embeddings', Get_embeddings()),
                 ('xgb', xgb)])

In [ ]:
pipe.fit(X, y)

In [ ]:
with open('xgb_model.pkl', 'wb') as output:
    pickle.dump(pipe, output)